In [55]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math, decimal 
import requests # rquests is a module that allows us to make HTTP requests to external APIs
dec = decimal.Decimal # Decimal tells Python to use a specific number of decimal places
import hvplot.pandas, holoviews as hv
import json
import streamlit as st # Streamlit is a web framework for writing beautiful and interactive web apps

def get_crypto_price(symbol, exchange, days): # get_crypto_price is a function that takes in a cryptocurrency symbol, exchange, and number of days and returns the price of the cryptocurrency over the past number of days
    api_key = 'YOUR API KEY'
    api_url = f'https://min-api.cryptocompare.com/data/v2/histoday?fsym={symbol}&tsym={exchange}&limit={days}&api_key={api_key}'
    raw = requests.get(api_url).json()
    df = pd.DataFrame(raw['Data']['Data'])[['time', 'close']].set_index('time') # here we are extracting the time and close price from the json file
    df.index = pd.to_datetime(df.index, unit = 's') # here we are converting the time index to a datetime object
    return df
FearGreedUrl = requests.get('https://api.alternative.me/fng/?limit=2').json() 

In [56]:
from pathlib import Path # here we import our data from the pathlib library
csvpath1 = Path("./full_moon.csv")
lunar_eclipse = Path("./lunar_eclipse.csv")
solar_eclipse = Path("./solar_eclipse.csv")
mercury_retro = Path("./mercury_retrograde.csv")

# Retrieve BTC API

In [57]:
btc = get_crypto_price('BTC', 'USD', 1825)
Price = btc['Price'] = btc['close']

# here we initialize some indicators that we will use to calculate the correlation between the price of the cryptocurrency and the lunar phase
SMA_9 = btc['9SMA'] = btc['close'].rolling(9).mean() # SMA = Simple Moving Average, SMA_9 = Simple Moving Average of the last 9 days
SMA_44 = btc['44SMA'] = btc['close'].rolling(44).mean()
SMA_117 = btc['117SMA'] = btc['close'].rolling(117).mean()
EMA_117 = btc['117EMA'] = btc['close'].ewm(117).mean() # EMA = Exponential Moving Average
btc = btc.reset_index() # here we reset the index of the dataframe so that we can use the index as a date and dfs have consistant indexing
btc = btc.rename(columns={'time':'dtime'}) # create two new columns, one for the time and one for the date
btc.set_index(['dtime'], inplace=True) # set the index to the date
btc_api = btc.drop(['close'], axis=1) # drop the close column from the dataframe on axis 1 or columns
btc_api

,Price,9SMA,44SMA,117SMA,117EMA
dtime,,,,,
2017-07-01,2424.61,NaN,NaN,NaN,2424.610000
2017-07-02,2536.46,NaN,NaN,NaN,2480.772979
2017-07-03,2572.47,NaN,NaN,NaN,2511.599153
2017-07-04,2617.32,NaN,NaN,NaN,2538.367725
2017-07-05,2627.86,NaN,NaN,NaN,2556.572125
...,...,...,...,...,...
2022-06-26,21031.85,20615.324444,26983.879545,35307.589829,37829.402926
2022-06-27,20718.16,20811.314444,26771.785227,35109.152137,37684.392367
2022-06-28,20251.96,20777.812222,26520.781818,34919.217009,37536.659862


### Plot BTC Price over Time

In [58]:
BTC = btc_api.hvplot.line( # here we are plotting the price of the cryptocurrency over time
    x = 'dtime',
    y = 'Price',
    xlabel = 'Date',
    ylabel = 'Price',
    title = 'BTC Price',
    legend = True
)
BTC

:Curve   [dtime]   (Price)

# Clean Lunar Data


In [59]:
#Create a Full Moon Dataframe
full_moon = pd.read_csv(csvpath1, parse_dates=True, index_col=' Date', infer_datetime_format=True) # here we are reading in the csv file and parsing the date column as a datetime object
full_moon.reset_index(inplace=True) # here we are resetting the index of the dataframe so that we can use the index as a date and dfs have consistant indexing
full_moon = full_moon.rename(columns = {' Date':'Full_Moon'}) # here we are renaming the column name to Full_Moon with a dictionary populated with the column name and the new name
full_moon.drop([' Time'], axis=1, inplace=True) # dropp the Time column from the dataframe on axis 1 or columns

In [60]:
# Create a Moon Data table with the BTC Price data for each Full and New Moon date

full_moon = full_moon.join(btc['Price'], on = 'Full_Moon', how = 'inner') # here we are joining the full moon dataframe with the BTC price dataframe on the Full_Moon column
full_moon['Full Moon Price'] = full_moon['Price']   # here we are creating a new column called Full Moon Price that contains the price of the cryptocurrency at the Full Moon date
full_moon = full_moon.drop(columns=['Price'])    # here we are dropping the Price column from the dataframe

# full_moon

In [61]:
#Create a New Moon df
new_moon = pd.DataFrame(full_moon['Full_Moon'] + timedelta(days=15)) # creating a new dataframe that contains the date of the New Moon
new_moon.rename(columns={'Full_Moon':'New_Moon'}, inplace=True) # renaming the column name to New_Moon with a dictionary populated with the column name and the new name
new_moon.drop(new_moon.index[-1], inplace=True) # dropping the last row from the dataframe. index[-1] is the last row in the dataframe
new_moon = new_moon.rename(columns = {'Full_Moon':'New_Moon'}) # renaming the column name to New_Moon with a dictionary populated with the column name and the new name
new_moon = new_moon.join(btc['Price'], on = 'New_Moon', how = 'inner')  # here we are joining the new moon dataframe with the BTC price dataframe on the New_Moon column. on = 'New_Moon' is the column that we are joining on, how = 'inner' is the type of join
new_moon['New Moon Price'] = new_moon['Price'] # here we are creating a new column called New Moon Price that contains the price of the cryptocurrency at the New Moon date
new_moon = new_moon.drop(columns=['Price']) # here we are dropping the Price column from the dataframe since New Moon Price has been created with the data from Price column and we don't need the Price column anymore

# new_moon

In [62]:
moon_data = pd.concat([new_moon,full_moon], axis=1, join='inner') # concatenating the new moon dataframe with the full moon dataframe on the New_Moon column

In [63]:
lunar_eclipse_df = pd.read_csv(lunar_eclipse, parse_dates=True, infer_datetime_format=True) # here we are creating a dataframe from the lunar eclipse csv file
lunar_eclipse_df.rename(columns={'Date':'lunar eclipse'}, inplace=True) # renaming the column name to lunar eclipse with a dictionary populated with the column name and the new name
lunar_eclipse_df['lunar eclipse'] = pd.to_datetime(lunar_eclipse_df['lunar eclipse']) # here we are converting the lunar eclipse column to a datetime object
lunar_eclipse_df = lunar_eclipse_df.join(btc['Price'], on = 'lunar eclipse', how = 'inner') # here we are joining the lunar eclipse dataframe with the BTC price dataframe on the lunar eclipse column. on = 'lunar eclipse' is the column that we are joining on, how = 'inner' is the type of join

In [64]:
solar_eclipse_df = pd.read_csv(solar_eclipse, parse_dates=True, infer_datetime_format=True) # creating a dataframe from the solar eclipse csv file
solar_eclipse_df.rename(columns={'Date':'solar eclipse'}, inplace=True) # renaming the column name to solar eclipse with a dictionary populated with the column name and the new name
solar_eclipse_df['solar eclipse'] = pd.to_datetime(solar_eclipse_df['solar eclipse']) # here we are converting the solar eclipse column to a datetime object
solar_eclipse_df.drop(['Unnamed: 1'], axis = 1, inplace=True) # dropping the Unnamed: 1 column from the dataframe on axis 1 or columns

solar_eclipse_df = solar_eclipse_df.join(btc['Price'], on = 'solar eclipse', how = 'inner') # here we are joining the solar eclipse dataframe with the BTC price dataframe on the solar eclipse column. on = 'solar eclipse' is the column that we are joining on, how = 'inner' is the type of join

In [65]:
mercury_data = pd.read_csv(mercury_retro, parse_dates=True, infer_datetime_format=True) # creating a dataframe from the mercury retrograde csv file
mercury_data['Start'] = pd.to_datetime(mercury_data['Start'])  # here we are converting the Start column to a datetime object
mercury_data['End'] = pd.to_datetime(mercury_data['End']) # here we are converting the End column to a datetime object
mercury_data['Term'] = mercury_data['End'] - mercury_data['Start']  # here we are creating a new column called Term that contains the difference between the End and Start dates
# mercury_data = pd.date_range(start=mercury_data['Start'], end=mercury_data['End'])
start = mercury_data['Start']
start_hv = start.hvplot.scatter()
end = mercury_data['End']
end_hv = end.hvplot.scatter()

# mercury_data

# start_hv*end_hv

# Plot Lunar Data over BTC Data

In [66]:
# create hvplot figures to then overaly 
glyph_1 = btc['Price'].hvplot.line( 
    'dtime', 'Price',
    color='#e7e7e7',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_2 = full_moon.hvplot.scatter(
    x = 'Full_Moon',
    y = 'Full Moon Price',
    color='#ffcd33',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_3 = new_moon.hvplot.scatter(
    x = 'New_Moon',
    y = 'New Moon Price',
    color='#ff6533',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_4 = btc['9SMA'].hvplot.line(
    'dtime', '9SMA',
    color='#70eac4',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_9 = btc['44SMA'].hvplot.line(
    'dtime', '44SMA',
    color='pink',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_5 = btc['117SMA'].hvplot.line(
    'dtime', '117SMA',
    color='#55d24a',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_6 = btc['117EMA'].hvplot.line(
    'dtime', '117EMA',
    color='#22a91a',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_7 = lunar_eclipse_df.hvplot.scatter(
    x = 'lunar eclipse',
    y = 'Price',
    color='#505050',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_8 = solar_eclipse_df.hvplot.scatter(
    x = 'solar eclipse',
    y = 'Price',
    color='#000000',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

btc_time = glyph_1*glyph_2*glyph_3*glyph_4*glyph_5*glyph_6*glyph_9*glyph_7*glyph_8
btc_time

:Overlay
   .Curve.Price    :Curve   [dtime]   (Price)
   .Scatter.I      :Scatter   [Full_Moon]   (Full Moon Price)
   .Scatter.II     :Scatter   [New_Moon]   (New Moon Price)
   .Curve.A_9SMA   :Curve   [dtime]   (9SMA)
   .Curve.A_117SMA :Curve   [dtime]   (117SMA)
   .Curve.A_117EMA :Curve   [dtime]   (117EMA)
   .Curve.A_44SMA  :Curve   [dtime]   (44SMA)
   .Scatter.III    :Scatter   [lunar eclipse]   (Price)
   .Scatter.IV     :Scatter   [solar eclipse]   (Price)

## Prepare Lunar and BTC Data for Merge

In [67]:
# here we initialize some indicators simlar to the ones in the previous section, but with some different parameters
btc = get_crypto_price('BTC', 'USD', 1825)
Price = btc['Price'] = btc['close']
SMA_8 = btc['8SMA'] = btc['close'].rolling(8).mean()
SMA_55 = btc['55SMA'] = btc['close'].rolling(55).mean()
SMA_200 = btc['200SMA'] = btc['close'].rolling(200).mean()
EMA_200 = btc['200EMA'] = btc['close'].ewm(200).mean()
btc = btc.reset_index()
btc = btc.rename(columns={'time':'dtime'})
# btc.set_index(['dtime'], inplace=True)
btc_api = btc.drop(['close'], axis=1)

In [68]:
# Read Full_Moon CSV, Clean index, infer Datetime
full_moon = pd.read_csv(csvpath1) # creating a dataframe from the full moon csv file
full_moon = full_moon.rename(columns = {' Date':'dtime'}) # renaming the column name to dtime with a dictionary populated with the column name and the new name
full_moon['dtime'] = full_moon['dtime']+' '+full_moon[' Time']  # here we are adding the time to the date. this is done by concatenating the date and the time and 
                                                                # the space between the two is created with + and then an empty string followed by another +
full_moon['Phase'] = full_moon.loc['dtime',:] = 1 # here we are creating a new column called Phase that contains the value 1
full_moon.drop([' Time'], axis=1, inplace=True) # dropping the Time column from the dataframe on axis 1 or columns
full_moon.drop(full_moon.index[-1], inplace=True) # dropping the last row from the dataframe

full_moon['dtime'] = pd.to_datetime(full_moon['dtime'], errors='coerce', format='%Y-%m-%d %H:%M:%S') # here we are converting the dtime column to a datetime object some formatting. 
                                                                                                     # format='%Y-%m-%d %H:%M:%S') gives us a format of YYYY-MM-DD HH:MM:SS
full_moon['dtime'] = pd.DataFrame(full_moon['dtime'] + timedelta(hours=-2)) # here we are adding 2 hours to the dtime column to make it UTC time. 
                                                                            # timedelta(hours=-2) is a timedelta object that adds 2 hours to the dtime column

full_moon

,dtime,Phase
0,2017-11-04 04:22:55,1.0
1,2017-12-03 14:46:59,1.0
2,2018-01-02 01:24:05,1.0
3,2018-01-31 12:26:44,1.0
4,2018-03-01 23:51:21,1.0
...,...,...
406,2050-09-01 08:30:54,1.0
407,2050-09-30 16:31:48,1.0
408,2050-10-30 02:16:00,1.0
409,2050-11-28 14:09:48,1.0


In [69]:
# Timedelta + 15 days to create New_Moon Dataframe
new_moon = pd.DataFrame(full_moon['dtime'] + timedelta(days=14, hours=19, minutes=26))  # this is creating a new dataframe with the dtime column + 14 days + 19 hours + 26 minutes
                                                                                        # this is the amount of time between full moon and new moon 
new_moon['Phase'] = new_moon.loc['dtime',:] = -1 # here we are creating a new column called Phase that contains the value -1. We are saying the new moon is the negative phase of the full moon
new_moon.drop(new_moon.index[-1], inplace=True) # dropping the last row from the dataframe
new_moon['dtime'] = pd.to_datetime(new_moon['dtime']) # here we are converting the dtime column to a datetime object some formatting

new_moon

,dtime,Phase
0,2017-11-18 23:48:55,-1.0
1,2017-12-18 10:12:59,-1.0
2,2018-01-16 20:50:05,-1.0
3,2018-02-15 07:52:44,-1.0
4,2018-03-16 19:17:21,-1.0
...,...,...
406,2050-09-16 03:56:54,-1.0
407,2050-10-15 11:57:48,-1.0
408,2050-11-13 21:42:00,-1.0
409,2050-12-13 09:35:48,-1.0


In [70]:
full_moon['dtime'] = pd.to_datetime(full_moon['dtime']).dt.date # converting the dtime column to a datetime object some 
                                                                # formatting and then converting it to a date of the dtime column in the full moon dataframe
new_moon['dtime'] = pd.to_datetime(new_moon['dtime']).dt.date # converting the dtime column to a datetime object some formatting of the new moon dataframe

In [71]:
# Append Lunar Dataframes to create one table sorted by 'dtime'
phase_data = full_moon.append(new_moon)
phase_data.sort_values('dtime', inplace= True)
phase_data.reset_index(inplace=True)
phase_data.drop(['index'], axis = 1, inplace=True)
phase_data['dtime'] = pd.to_datetime(phase_data['dtime'])
# phase_data.set_index(['dtime'], inplace=True)
phase_data

,dtime,Phase
0,2017-11-04,1.0
1,2017-11-18,-1.0
2,2017-12-03,1.0
3,2017-12-18,-1.0
4,2018-01-02,1.0
...,...,...
817,2050-11-13,-1.0
818,2050-11-28,1.0
819,2050-12-13,-1.0
820,2050-12-28,1.0


In [72]:
phase_df = phase_data.copy() # creating a copy of the phase_data dataframe to use in the next section

In [73]:
eclipse_df1 = pd.read_csv(lunar_eclipse, parse_dates=True, infer_datetime_format=True) # creating a dataframe from the lunar eclipse csv file
eclipse_df1.rename(columns={'Date':'dtime'}, inplace=True)  # renaming the column name to dtime with a dictionary populated with the column name and the new name
eclipse_df1['Eclipse'] = 1  # here we are creating a new column called Eclipse that contains the value 1
eclipse_df1['dtime'] = pd.to_datetime(eclipse_df1['dtime']) # here we are converting the dtime column to a datetime object some formatting
# eclipse_df1

In [74]:
eclipse_df2 = pd.read_csv(solar_eclipse, parse_dates=True, infer_datetime_format=True) # creating a dataframe from the solar eclipse csv file
eclipse_df2.rename(columns={'Date':'dtime'}, inplace=True) # renaming the column name to dtime with a dictionary populated with the column name and the new name
eclipse_df2['Eclipse'] = -1 # here we are creating a new column called Eclipse that contains the value -1 
eclipse_df2['dtime'] = pd.to_datetime(eclipse_df2['dtime']) # here we are converting the dtime column to a datetime object some formatting
eclipse_df2.drop(['Unnamed: 1'], axis = 1, inplace=True) # dropping the Unnamed: 1 column from the dataframe
# eclipse_df2

In [75]:
eclipse_data = eclipse_df1.append(eclipse_df2) # appending the two dataframes to create one table sorted by 'dtime'
eclipse_data.sort_values('dtime', inplace= True) # sorting the dataframe by the dtime column
eclipse_data.reset_index(inplace=True) # resetting the index
eclipse_data.drop(['index'], axis = 1, inplace=True) # dropping the index column from the dataframe
eclipse_data['dtime'] = pd.to_datetime(eclipse_data['dtime']) # here we are converting the dtime column to a datetime object some formatting
eclipse_data.columns 
eclipse_data.tail(365)

,dtime,Eclipse
95,2020-11-30,1
96,2020-12-14,-1
97,2021-05-26,1
98,2021-06-10,-1
99,2021-11-19,1
...,...,...
455,2099-09-29,1
456,2100-02-24,1
457,2100-03-10,-1
458,2100-08-19,1


In [76]:
phase_data = pd.merge_asof(phase_data,eclipse_data,on='dtime',tolerance=pd.Timedelta('1day'),allow_exact_matches=True) # merging the phase_data dataframe with the eclipse_data dataframe
# phase_data

In [77]:
# creating a function that takes in two arguments x and y. if x is greater than y then return 1 if y is greater than x then return -1.  
# these 1 and -1s will be used as an indicator to buy or sell 
def Tru_Moo(x, y): 
    if x - y > 0:
        return 1
    return -1

 # here we are creating a new column called Tru_Moo then setting it equal to the result of the lambda function. The lambda functions a short hand way to create a function.
moon_data['Tru_Moo'] = moon_data.apply(lambda row: Tru_Moo(row['New Moon Price'], row['Full Moon Price']), axis=1)  
 
moon_data['PCT Change'] = (moon_data['New Moon Price'] - moon_data['Full Moon Price']) / moon_data['New Moon Price']    # creating a new column called PCT Change that is the percentage 
                                                                                                                        # change between the new moon price and full moon price
 

#for ind in (moon_data.index):
#print(moon_data.iloc[:,2])
#print(moon_data.iloc[1:,3]- moon_data.iloc[:,2])

moon_data = moon_data.rename(columns={'New_Moon':'dtime','PCT Change':'moo_pct'})  # renaming the column name to dtime with a dictionary populated with the column name and the new name
moon_data = moon_data.append(full_moon).reset_index() # appending the full moon dataframe to the moon_data dataframe
moon_data['dtime'] = pd.to_datetime(moon_data['dtime']) # here we are converting the dtime column to a datetime object some formatting
moon_data.sort_values('dtime', inplace= True) # sorting the dataframe by the dtime column
moon_data = pd.merge_asof(phase_data,moon_data,on='dtime',tolerance=pd.Timedelta('1day'),allow_exact_matches=True) # this merge is to merge the phase_data dataframe with the moon_data dataframe. tolerance is the amount of time between the two dataframes
moon_data.drop(['New Moon Price','Full_Moon','Full Moon Price','index','Eclipse','Phase_x','Phase_y'], axis=1, inplace=True) # dropping the columns that are not needed
moon_data.set_index('dtime', inplace=True) # setting the index to the dtime column
moon_data['Tru_Moo'] = moon_data['Tru_Moo'].fillna(0) # filling the Tru_Moo column with 0's
moon_data['moo_pct'] = moon_data['moo_pct'].fillna(0) # filling the moo_pct column with 0's
moon_data

,Tru_Moo,moo_pct
dtime,,
2017-11-04,0.0,0.000000
2017-11-18,0.0,0.000000
2017-12-03,0.0,0.000000
2017-12-18,1.0,0.407231
2018-01-02,0.0,0.000000
...,...,...
2050-11-13,0.0,0.000000
2050-11-28,0.0,0.000000
2050-12-13,0.0,0.000000


# Merge Dataframes on 'dtime'

In [78]:

moon_merge = phase_data.merge(btc_api, on='dtime', how='outer') # merging the phase_data dataframe with the btc_api dataframe
moon_merge['dtime'] = pd.to_datetime(moon_merge['dtime']) # here we are converting the dtime column to a datetime object some formatting
moon_merge['Eclipse'] = moon_merge['Eclipse'].fillna(0) # filling the Eclipse column with 0's
moon_merge['200SMA'] = moon_merge['200SMA'].fillna(0) # filling the 200SMA column with 0's
moon_merge['200SMA'] = moon_merge['200SMA'].fillna(0) # filling the 200SMA column with 0's
moon_merge['PCT Change'] = moon_merge['Price'].pct_change() # creating a new column called PCT Change that is the percentage change between the new moon price and full moon price
moon_merge.set_index(['dtime'], inplace=True)
moon_merge

,Phase,Eclipse,Price,8SMA,55SMA,200SMA,200EMA,PCT Change
dtime,,,,,,,,
2017-11-04,1.0,0.0,7363.80,6591.02000,4947.186545,0.00000,4263.931643,NaN
2017-11-18,-1.0,0.0,7780.91,6994.40750,5772.459636,0.00000,4634.916241,0.056643
2017-12-03,1.0,0.0,11246.21,10220.33625,7131.205455,0.00000,5258.595898,0.445359
2017-12-18,-1.0,0.0,18972.32,17694.93375,9989.369636,0.00000,6625.232930,0.686997
2018-01-02,1.0,0.0,14754.13,14318.17500,12186.421273,0.00000,7580.972900,-0.222334
...,...,...,...,...,...,...,...,...
2022-06-25,NaN,0.0,21474.19,20563.25875,28590.999273,38656.71040,38935.839286,0.012005
2022-06-26,NaN,0.0,21031.85,20822.95875,28273.160000,38509.30445,38846.754637,-0.020599
2022-06-27,NaN,0.0,20718.16,20843.54375,27963.937818,38374.92600,38756.552474,-0.014915


In [79]:
test_df = moon_data.merge(moon_merge, on='dtime', how='inner') # merging the moon_data dataframe with the moon_merge dataframe
test_df = test_df.dropna()
test_df

,Tru_Moo,moo_pct,Phase,Eclipse,Price,8SMA,55SMA,200SMA,200EMA,PCT Change
dtime,,,,,,,,,,
2017-11-18,0.0,0.000000,-1.0,0.0,7780.91,6994.40750,5772.459636,0.00000,4634.916241,0.056643
2017-12-03,0.0,0.000000,1.0,0.0,11246.21,10220.33625,7131.205455,0.00000,5258.595898,0.445359
2017-12-18,1.0,0.407231,-1.0,0.0,18972.32,17694.93375,9989.369636,0.00000,6625.232930,0.686997
2018-01-02,0.0,0.000000,1.0,0.0,14754.13,14318.17500,12186.421273,0.00000,7580.972900,-0.222334
2018-01-16,0.0,0.000000,-1.0,0.0,11282.49,13666.68250,14054.222545,7098.68290,8332.979805,-0.235300
...,...,...,...,...,...,...,...,...,...,...
2022-05-01,-1.0,-0.049688,-1.0,-1.0,38480.53,38968.48000,41745.827091,47271.14055,42306.806700,-0.047336
2022-05-16,0.0,0.000000,1.0,1.0,29838.50,29931.58750,39784.711455,45151.20030,41642.423045,-0.224582
2022-05-30,0.0,0.000000,-1.0,0.0,31716.41,29522.77625,35674.930909,42802.09040,40830.173841,0.062936


### Machine Learning

In [80]:
test_df["signal"] = 0.0 # initializing the test dataframe with a column called signal that contains 0's 
#Buy Signal
test_df.loc[(test_df['Phase'] == 1), "signal"] = 1 # if the Phase column is 1 then the signal is to buy
#Sell Signal
test_df.loc[(test_df['Phase'] == -1) & (test_df['PCT Change'] > 0), "signal"] = -1 # if the Phase column is -1 and the PCT Change column is greater than 0 then the signal is to sell
test_df

,Tru_Moo,moo_pct,Phase,Eclipse,Price,8SMA,55SMA,200SMA,200EMA,PCT Change,signal
dtime,,,,,,,,,,,
2017-11-18,0.0,0.000000,-1.0,0.0,7780.91,6994.40750,5772.459636,0.00000,4634.916241,0.056643,-1.0
2017-12-03,0.0,0.000000,1.0,0.0,11246.21,10220.33625,7131.205455,0.00000,5258.595898,0.445359,1.0
2017-12-18,1.0,0.407231,-1.0,0.0,18972.32,17694.93375,9989.369636,0.00000,6625.232930,0.686997,-1.0
2018-01-02,0.0,0.000000,1.0,0.0,14754.13,14318.17500,12186.421273,0.00000,7580.972900,-0.222334,1.0
2018-01-16,0.0,0.000000,-1.0,0.0,11282.49,13666.68250,14054.222545,7098.68290,8332.979805,-0.235300,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-01,-1.0,-0.049688,-1.0,-1.0,38480.53,38968.48000,41745.827091,47271.14055,42306.806700,-0.047336,0.0
2022-05-16,0.0,0.000000,1.0,1.0,29838.50,29931.58750,39784.711455,45151.20030,41642.423045,-0.224582,1.0
2022-05-30,0.0,0.000000,-1.0,0.0,31716.41,29522.77625,35674.930909,42802.09040,40830.173841,0.062936,-1.0


Do not buy on full moon before eclipse.
When the 8 SMA > 200 SMA > 200 EMA = Bearish Cross, do not buy on full moon.
When the 8 SMA < 200 SMA > 200 EMA = Buy the full moon

In [81]:
X = test_df[['8SMA','55SMA','200SMA','Phase','Eclipse','Tru_Moo','PCT Change']] # creating a dataframe that contains the columns that we want to use as our features

display(X.head())
display(X.tail())

,8SMA,55SMA,200SMA,Phase,Eclipse,Tru_Moo,PCT Change
dtime,,,,,,,
2017-11-18,6994.40750,5772.459636,0.0000,-1.0,0.0,0.0,0.056643
2017-12-03,10220.33625,7131.205455,0.0000,1.0,0.0,0.0,0.445359
2017-12-18,17694.93375,9989.369636,0.0000,-1.0,0.0,1.0,0.686997
2018-01-02,14318.17500,12186.421273,0.0000,1.0,0.0,0.0,-0.222334
2018-01-16,13666.68250,14054.222545,7098.6829,-1.0,0.0,0.0,-0.235300


,8SMA,55SMA,200SMA,Phase,Eclipse,Tru_Moo,PCT Change
dtime,,,,,,,
2022-05-01,38968.48000,41745.827091,47271.14055,-1.0,-1.0,-1.0,-0.047336
2022-05-16,29931.58750,39784.711455,45151.20030,1.0,1.0,0.0,-0.224582
2022-05-30,29522.77625,35674.930909,42802.09040,-1.0,0.0,0.0,0.062936
2022-06-14,27502.26750,32291.415818,40488.62110,1.0,0.0,0.0,-0.302621
2022-06-29,20730.54375,27311.556909,38093.70890,-1.0,0.0,0.0,-0.091517


In [82]:
y = test_df["signal"] # creating a dataframe that contains the column that we want to use as our target
y

dtime
2017-11-18   -1.0
2017-12-03    1.0
2017-12-18   -1.0
2018-01-02    1.0
2018-01-16    0.0
             ... 
2022-05-01    0.0
2022-05-16    1.0
2022-05-30   -1.0
2022-06-14    1.0
2022-06-29    0.0
Name: signal, Length: 115, dtype: float64

In [83]:
# Select the start of the training period
training_begin = X.index.min() # the start of the training period is the first index in the dataframe

# Display the training begin date
print(training_begin) 

2017-11-18 00:00:00


In [84]:
from pandas.tseries.offsets import DateOffset
# Select the ending period for the training data with an offset of 4 years
training_end = X.index.min()+ DateOffset(years=4) # the end of the training period is the first index in the dataframe plus 4 years

# Display the training end date
print(training_end)

2021-11-18 00:00:00


In [85]:
# Generate the X_train and y_train DataFrames
X_train = X.loc[training_begin:training_end] # creating a training dataframe that contains the columns that we want to use as our features
y_train = y.loc[training_begin:training_end] # creating a training dataframe that contains the column that we want to use as our target

In [86]:
# Generate the X_test and y_test DataFrames
X_test = X.loc[training_end:] # creating a testing dataframe that contains the columns that we want to use as our features
y_test = y.loc[training_end:] # creating a testing dataframe that contains the column that we want to use as our target

In [87]:
from sklearn.preprocessing import StandardScaler

In [88]:
# Create a StandardScaler instance
scaler = StandardScaler() # creating a StandardScaler instance. StandardScaler is a class that normalizes the data
 
# Apply the scaler model to fit the X-train data
X_scaler = scaler.fit(X_train) # applying the scaler model to the X_train data
 
# Transform the X_train and X_test DataFrames using the X_scaler
X_train_scaled = X_scaler.transform(X_train) # X_scaler.transorm tells python to transorm the X_train data using the scaler model so that it is normalized
X_test_scaled = X_scaler.transform(X_test) # X_scaler.transorm tells python to transorm the X_test data using the scaler model so that it is normalized

In [89]:
from sklearn import svm
from sklearn.metrics import classification_report

In [90]:
# creating a svm model which is Support Vector Machine
# SVMs are a type of supervised learning methods that are used for classification, regression and outliers detection 

svm_model = svm.SVC() 
 
# Fit the model to the data using X_train_scaled and y_train
svm_model = svm_model.fit(X_train_scaled, y_train)

# Use the trained model to predict the trading signals for the training data
training_signal_predictions = svm_model.predict(X_train_scaled)

# Display the sample predictions
training_signal_predictions[:10]

array([-1.,  1., -1.,  1.,  0.,  1.,  0.,  1.,  0.,  1.])

In [91]:
training_report = classification_report(y_train, training_signal_predictions)

# Display the report
print(training_report)

              precision    recall  f1-score   support

        -1.0       1.00      1.00      1.00        26
         0.0       1.00      1.00      1.00        24
         1.0       1.00      1.00      1.00        49

    accuracy                           1.00        99
   macro avg       1.00      1.00      1.00        99
weighted avg       1.00      1.00      1.00        99



In [92]:
testing_signal_predictions = svm_model.predict(X_test_scaled)
testing_signal_predictions.shape

(16,)

In [93]:
testing_report = classification_report(y_test, testing_signal_predictions)

# Display the report
print(testing_report)

              precision    recall  f1-score   support

        -1.0       0.75      1.00      0.86         3
         0.0       1.00      0.80      0.89         5
         1.0       1.00      1.00      1.00         8

    accuracy                           0.94        16
   macro avg       0.92      0.93      0.92        16
weighted avg       0.95      0.94      0.94        16



In [94]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression(random_state = 0)
model.fit(X_train_scaled, y_train)
lr_pred = model.predict(X_test_scaled)
print(classification_report(y_test, lr_pred))

              precision    recall  f1-score   support

        -1.0       1.00      1.00      1.00         3
         0.0       1.00      1.00      1.00         5
         1.0       1.00      1.00      1.00         8

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [95]:
# Create a predictions DataFrame
predictions_df = pd.DataFrame(index=X_test.index) 

# Add the SVM model predictions to the DataFrame
predictions_df['Logistic_Prediction'] = lr_pred
predictions_df['SVM_Prediction'] = testing_signal_predictions

# predictions_df['Control'] = y_test
# Add the actual returns to the DataFrame
predictions_df['PCT Change'] = moon_merge['PCT Change']

# Add the strategy returns to the DataFrame
predictions_df['Logistic_Returns'] = predictions_df['PCT Change']*predictions_df['Logistic_Prediction']

predictions_df['SVM_Returns'] = predictions_df['PCT Change']*predictions_df['SVM_Prediction']

# Review the DataFrame
display(predictions_df.head())
display(predictions_df.tail()) 

,Logistic_Prediction,SVM_Prediction,PCT Change,Logistic_Returns,SVM_Returns
dtime,,,,,
2021-11-19,1.0,1.0,-0.053903,-0.053903,-0.053903
2021-12-04,0.0,0.0,-0.152965,-0.000000,-0.000000
2021-12-19,1.0,1.0,-0.051708,-0.051708,-0.051708
2022-01-02,-1.0,-1.0,0.013047,-0.013047,-0.013047
2022-01-17,1.0,1.0,-0.107428,-0.107428,-0.107428


,Logistic_Prediction,SVM_Prediction,PCT Change,Logistic_Returns,SVM_Returns
dtime,,,,,
2022-05-01,0.0,0.0,-0.047336,-0.000000,-0.000000
2022-05-16,1.0,1.0,-0.224582,-0.224582,-0.224582
2022-05-30,-1.0,-1.0,0.062936,-0.062936,-0.062936
2022-06-14,1.0,1.0,-0.302621,-0.302621,-0.302621
2022-06-29,0.0,0.0,-0.091517,-0.000000,-0.000000


In [96]:
# Plot the actual returns versus the strategy returns
(1 + predictions_df[['PCT Change', 'Logistic_Returns','SVM_Returns']]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [dtime]   (value)

In [97]:
test_df

,Tru_Moo,moo_pct,Phase,Eclipse,Price,8SMA,55SMA,200SMA,200EMA,PCT Change,signal
dtime,,,,,,,,,,,
2017-11-18,0.0,0.000000,-1.0,0.0,7780.91,6994.40750,5772.459636,0.00000,4634.916241,0.056643,-1.0
2017-12-03,0.0,0.000000,1.0,0.0,11246.21,10220.33625,7131.205455,0.00000,5258.595898,0.445359,1.0
2017-12-18,1.0,0.407231,-1.0,0.0,18972.32,17694.93375,9989.369636,0.00000,6625.232930,0.686997,-1.0
2018-01-02,0.0,0.000000,1.0,0.0,14754.13,14318.17500,12186.421273,0.00000,7580.972900,-0.222334,1.0
2018-01-16,0.0,0.000000,-1.0,0.0,11282.49,13666.68250,14054.222545,7098.68290,8332.979805,-0.235300,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-01,-1.0,-0.049688,-1.0,-1.0,38480.53,38968.48000,41745.827091,47271.14055,42306.806700,-0.047336,0.0
2022-05-16,0.0,0.000000,1.0,1.0,29838.50,29931.58750,39784.711455,45151.20030,41642.423045,-0.224582,1.0
2022-05-30,0.0,0.000000,-1.0,0.0,31716.41,29522.77625,35674.930909,42802.09040,40830.173841,0.062936,-1.0


In [98]:
#BUY
entry = test_df[test_df['signal'] == 1.0].hvplot.scatter(
    'dtime','Price',
    color='green',
    marker='^',
    size=100,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

#SELL
exit = test_df[test_df['signal'] == -1.0].hvplot.scatter(
    'dtime','Price',
    color='red',
    marker='v',
    size=100,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
btc_price = test_df[['Price']].hvplot(
    'dtime','Price',
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = test_df[['8SMA', '55SMA', '200SMA', '200EMA']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Create the overlay plot
entry_exit_plot = btc_price * moving_avgs * entry * exit

# Show the plot with a title
entry_exit_plot.opts(
    title="BTC - Entry and Exit Points"
)

:Overlay
   .Curve.I     :Curve   [dtime]   (Price)
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [dtime]   (value)
   .Scatter.I   :Scatter   [dtime]   (Price)
   .Scatter.II  :Scatter   [dtime]   (Price)

In [99]:
# Set initial capital
initial_capital = float(20000)

# Set the share size
share_size = 1

In [100]:
test_df['Position'] = share_size * test_df['signal']

In [101]:
# Determine the points in time where a share position is bought or sold
test_df['Entry/Exit Position'] = test_df['Position'].diff()

In [102]:
# Multiply the close price by the number of shares held, or the Position
test_df['Portfolio Holdings'] = test_df['Price'] * test_df['Position']

In [103]:
# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
test_df['Portfolio Cash'] = initial_capital - (test_df['Price'] * test_df['Entry/Exit Position']).cumsum() 

In [104]:
# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
test_df['Portfolio Total'] = test_df['Portfolio Cash'] + test_df['Portfolio Holdings']

In [105]:
# Calculate the portfolio daily returns
test_df['Portfolio Cycle Returns'] = test_df['Portfolio Total'].pct_change()

In [106]:
# Calculate the portfolio cumulative returns
test_df['Portfolio Cumulative Returns'] = (1 + test_df['Portfolio Cycle Returns']).cumprod() - 1 

In [107]:
# Visualize entry position relative to total portfolio value
entry = test_df[test_df['signal'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize exit position relative to total portfolio value
exit = test_df[test_df['signal'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize the value of the total portfolio
total_portfolio_value = test_df[['Portfolio Total']].hvplot(
    line_color='black',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Visualize close price for the investment
btc_price = test_df[['Price']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit * btc_price
portfolio_entry_exit_plot.opts(
    title="Lunar Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [dtime]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [dtime]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [dtime]   (Portfolio Total)
   .Curve.II                   :Curve   [dtime]   (Price)